<a href="https://colab.research.google.com/github/kyochanpy/Kaggle_Indoor_Location_Navigation/blob/main/create_dataset/beacon_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Wifi features

This this is the code to generate the wifi features available in [this dataset](https://www.kaggle.com/devinanzelmo/indoor-navigation-and-location-wifi-features). Using these features can get a score below 14. For an example notebook using them see [this notebook](https://www.kaggle.com/devinanzelmo/wifi-features-lightgbm-starter). They only uses waypoints, wifi and timestamp data to generate solution. See this [forum post](https://www.kaggle.com/c/indoor-location-navigation/discussion/215445) for an outline of this solution method, and methods of improvement.

There are `break`'s inserted into loops which need to be removed to get this to run. Right now data is written to current working directory. This takes 2-4 hours to run depending on hard drive etc. There is a lot of room for improvement speeding up feature generation. 

**Update:** I added one line that creates a column for the path filename, this allows for a groupkfold crossvalidation. 


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
import pandas as pd
import numpy as np
import glob
import os
import gc
import json 
import shutil

In [12]:
base_path = '/content/drive/MyDrive/fixed_train/'

In [13]:
# pull out all the buildings actually used in the test set, given current method we don't need the other ones
ssubm = pd.read_csv('/content/drive/MyDrive/sample_submission.csv')

# only 24 of the total buildings are used in the test set, 
# this allows us to greatly reduce the intial size of the dataset

ssubm_df = ssubm["site_path_timestamp"].apply(lambda x: pd.Series(x.split("_")))
used_buildings = sorted(ssubm_df[0].value_counts().index.tolist())

# dictionary used to map the floor codes to the values used in the submission file. 
floor_map = {"B2":-2, "B1":-1, "F1":0, "F2": 1, "F3":2, "F4":3, "F5":4, "F6":5, "F7":6,"F8":7, "F9":8,
             "1F":0, "2F":1, "3F":2, "4F":3, "5F":4, "6F":5, "7F":6, "8F": 7, "9F":8}

In [15]:
used_buildings = ['5a0546857ecc773753327266',
 '5c3c44b80379370013e0fd2b',
 '5d27096c03f801723c31e5e0',
 '5d27097f03f801723c320d97',
 '5d27099f03f801723c32511d',
 '5d2709a003f801723c3251bf',
 '5d2709b303f801723c327472',
 '5d2709bb03f801723c32852c',
 '5d2709c303f801723c3299ee',
 '5d2709d403f801723c32bd39',
 '5d2709e003f801723c32d896',
 '5da138274db8ce0c98bbd3d2',
 '5da1382d4db8ce0c98bbe92e',
 '5da138314db8ce0c98bbf3a0',
 '5da138364db8ce0c98bc00f1',
 '5da1383b4db8ce0c98bc11ab',
 '5da138754db8ce0c98bca82f',
 '5da138764db8ce0c98bcaa46',
 '5da1389e4db8ce0c98bd0547',
 '5da138b74db8ce0c98bd4774',
 '5da958dd46f8266d0737457b',
 '5dbc1d84c1eb61796cf7c010',
 '5dc8cea7659e181adb076a3f']

In [ ]:
# get only the wifi bssid that occur over 1000 times(this number can be experimented with)
# these will be the only ones used when constructing features
mac_address = dict()

for building in used_buildings:
    folders = sorted(glob.glob(os.path.join(base_path,'train/'+building+'/*')))
    print(building)
    beacon = list()
    for folder in folders:
        floor = floor_map[folder.split('/')[-1]]
        files = glob.glob(os.path.join(folder, "*.txt"))
        for file in files:
            with open(file) as f:
                txt = f.readlines()
                for e, line in enumerate(txt):
                    tmp = line.strip().split()
                    if tmp[1] == "TYPE_BEACON":
                        beacon.append(tmp)
    df = pd.DataFrame(beacon)
    #top_bssid = df[3].value_counts().iloc[:500].index.tolist()
    value_counts = df[8].value_counts()
    top_mac_address = value_counts[value_counts > 0].index.tolist()
    print(len(top_mac_address))
    mac_address[building] = top_mac_address
    del df
    del beacon
    gc.collect()

5a0546857ecc773753327266


In [9]:
# generate all the training data 
building_dfs = dict()

for building in used_buildings:
    folders = sorted(glob.glob(os.path.join(base_path,'train', building +'/*')))
    dfs = list()
    index = sorted(mac_address[building])
    print(building)
    for folder in folders:
        floor = floor_map[folder.split('/')[-1]]
        files = glob.glob(os.path.join(folder, "*.txt"))
        print(floor)
        for file in files:
            beacon = list()
            waypoint = list()
            with open(file) as f:
                txt = f.readlines()
            for line in txt:
                line = line.strip().split()
                if line[1] == "TYPE_WAYPOINT":
                    waypoint.append(line)
                if line[1] == "TYPE_BEACON":
                    beacon.append(line)

            df = pd.DataFrame(np.array(beacon))    

            # generate a feature, and label for each wifi block
            for gid, g in df.groupby(0):
                dists = list()
                for e, k in enumerate(waypoint):
                    dist = abs(int(gid) - int(k[0]))
                    dists.append(dist)
                nearest_wp_index = np.argmin(dists)
                
                g = g.drop_duplicates(subset=8)
                tmp = g.iloc[:,[6,8]]
                feat = tmp.set_index(8).reindex(index).replace(np.nan, -999).T
                feat["x"] = float(waypoint[nearest_wp_index][2])
                feat["y"] = float(waypoint[nearest_wp_index][3])
                feat["f"] = floor
                feat["path"] = file.split('/')[-1].split('.')[0] # useful for crossvalidation
                feat["timestamp"] = gid
                dfs.append(feat)
                
    building_df = pd.concat(dfs)
    building_dfs[building] = df
    building_df.to_csv(building+"_train.csv")
    shutil.move(f'{building}_train.csv', '/content/drive/MyDrive/beacon_rssi')

5dc8cea7659e181adb076a3f
-1
0
1
2
3
4
5
6


In [8]:
# generate all the training data 
building_dfs = dict()

for building in used_buildings:
    folders = sorted(glob.glob(os.path.join(base_path,'train', building +'/*')))
    dfs = list()
    index = sorted(mac_address[building])
    print(building)
    for folder in folders:
        floor = floor_map[folder.split('/')[-1]]
        files = glob.glob(os.path.join(folder, "*.txt"))
        print(floor)
        for file in files:
            beacon = list()
            waypoint = list()
            with open(file) as f:
                txt = f.readlines()
            for line in txt:
                line = line.strip().split()
                if line[1] == "TYPE_WAYPOINT":
                    waypoint.append(line)
                if line[1] == "TYPE_BEACON":
                    beacon.append(line)

            df = pd.DataFrame(np.array(beacon))    

            # generate a feature, and label for each wifi block
            for gid, g in df.groupby(0):
                dists = list()
                for e, k in enumerate(waypoint):
                    dist = abs(int(gid) - int(k[0]))
                    dists.append(dist)
                nearest_wp_index = np.argmin(dists)
                
                g = g.drop_duplicates(subset=8)
                tmp = g.iloc[:,[7,8]]
                feat = tmp.set_index(8).reindex(index).replace(np.nan, -999).T
                feat["x"] = float(waypoint[nearest_wp_index][2])
                feat["y"] = float(waypoint[nearest_wp_index][3])
                feat["f"] = floor
                feat["path"] = file.split('/')[-1].split('.')[0] # useful for crossvalidation
                feat["timestamp"] = gid
                dfs.append(feat)
                
    building_df = pd.concat(dfs)
    building_dfs[building] = df
    building_df.to_csv(building+"_train.csv")
    shutil.move(f'{building}_train.csv', '/content/drive/MyDrive/beacon_distance')

5dc8cea7659e181adb076a3f
-1
0
1
2
3
4
5
6


Error: ignored

In [10]:
ｇ# Generate the features for the test set

ssubm_building_g = ssubm_df.groupby(0)
feature_dict = dict()

for gid0, g0 in ssubm_building_g:
    index = sorted(mac_address[g0.iloc[0,0]])
    feats = list()
    print(gid0)
    for gid,g in g0.groupby(1):

        # get all wifi time locations, 
        with open(os.path.join(base_path, 'test/' + g.iloc[0,1] + '.txt')) as f:
            txt = f.readlines()

        beacon = list()

        for line in txt:
            line = line.strip().split()
            if line[1] == "TYPE_BEACON":
                beacon.append(line)

        beacon_df = pd.DataFrame(beacon)
        beacon_points = pd.DataFrame(beacon_df.groupby(0).count().index.tolist())
        
        for timepoint in g.iloc[:,2].tolist():

            deltas = (beacon_points.astype(int) - int(timepoint)).abs()
            min_delta_idx = deltas.values.argmin()
            beacon_block_timestamp = beacon_points.iloc[min_delta_idx].values[0]
            
            beacon_block = beacon_df[beacon_df[0] == beacon_block_timestamp].drop_duplicates(subset=8)
            feat = beacon_block.set_index(8)[6].reindex(index).fillna(-999)

            feat['site_path_timestamp'] = g.iloc[0,0] + "_" + g.iloc[0,1] + "_" + timepoint
            feat['path'] = g.iloc[0,1]
            feat['timestamp'] = timepoint
            feats.append(feat)
    feature_df = pd.concat(feats, axis=1).T
    feature_df.to_csv(gid0+"_test.csv")
    feature_dict[gid0] = feature_df
    shutil.move(f'{gid0}_test.csv', '/content/drive/MyDrive/beacon_rssi')

KeyError: ignored

In [ ]:
ｇ# Generate the features for the test set

ssubm_building_g = ssubm_df.groupby(0)
feature_dict = dict()

for gid0, g0 in ssubm_building_g:
    index = sorted(mac_address[g0.iloc[0,0]])
    feats = list()
    print(gid0)
    for gid,g in g0.groupby(1):

        # get all wifi time locations, 
        with open(os.path.join(base_path, 'test/' + g.iloc[0,1] + '.txt')) as f:
            txt = f.readlines()

        beacon = list()

        for line in txt:
            line = line.strip().split()
            if line[1] == "TYPE_BEACON":
                beacon.append(line)

        beacon_df = pd.DataFrame(beacon)
        beacon_points = pd.DataFrame(beacon_df.groupby(0).count().index.tolist())
        
        for timepoint in g.iloc[:,2].tolist():

            deltas = (beacon_points.astype(int) - int(timepoint)).abs()
            min_delta_idx = deltas.values.argmin()
            beacon_block_timestamp = beacon_points.iloc[min_delta_idx].values[0]
            
            beacon_block = beacon_df[beacon_df[0] == beacon_block_timestamp].drop_duplicates(subset=8)
            feat = beacon_block.set_index(8)[7].reindex(index).fillna(-999)

            feat['site_path_timestamp'] = g.iloc[0,0] + "_" + g.iloc[0,1] + "_" + timepoint
            feat['path'] = g.iloc[0,1]
            feat['timestamp'] = timepoint
            feats.append(feat)
    feature_df = pd.concat(feats, axis=1).T
    feature_df.to_csv(gid0+"_test.csv")
    feature_dict[gid0] = feature_df
    shutil.move(f'{gid0}_test.csv', '/content/drive/MyDrive/beacon_distance')

In [ ]:
a = pd.read_csv('/content/drive/MyDrive/wifi_0_02/5a0546857ecc773753327266_train.csv')
a

,Unnamed: 0,000840e5c600de293cea57f13326f273c86c3988,0023c80b0e4cb8338cf25de26143bb0570a16471,005246b6f51feb1a069e8f005d3e6aba2591b65b,0076ff7a084cb2ac8c146139965ab1be296e72c4,0089ad1dd75b13e2c3ceda344988c9f89a83a2f9,009a3ed672be7bd1b9c4437b43a53296771af098,00ad587dcb9c7ce3788b92e22777a22ee0efea31,00af060fc145ee6a6a50475efa57b91cbf54237f,00bcc61bdea4d52d050822d66952dd707c2fcdf3,00c8933965e23cd2bb890ca08ee7f8a22f933df8,00e2787f25a1d770f62ed57d08b9393c7105b00d,00ef32991244ca6ce16c9ee3b910886401766bf6,00f0904087c01d922d6ebf3005607dfdeaf6687b,00f09c4561768a871af2921cffa0a4fda515766b,00f36a2e037357f98009f8d084fa70eb5071aa42,0102fdd6f12bee16a21bb62cd67afa00d2272670,011aac59a4af0f887f3914bf0dca6263625e9c5b,011e20ebf721a1c6dfec42e8ed1e2ac566073a2a,01506a39edc2da8d9003f0f12ce5dcc2e8c2772e,015504eea807d723f1d1a1aca43544189830aed7,01846fc4449a414413024fbc6d2e9be1a2203f3c,0192ce7fc9f7e169d3edd58151d50555d70ebb27,019f793ca026ac0c1489ba3a199984aa7322ac55,01a39206e3987e6708631b62b70ecfd55ab05a7d,01c325eeb378ebe2ab3212fe86bf0060f71ced22,01d2f676abab6ec03ec5dc696bfd49d66e392ea1,01da12eb27c6e37d03d691c31b40591a724db4a2,01dd00cd784a639f7d6609ddc8bfc6c9af3b47d3,01e25e4a25acd32baf5137b3031151f751fadbb4,01ead5ee97d0329b2c228fece5a96b0358a3cab1,0231e64a6307175a78f12cfe001c579e580ed721,0235185b1ffbe48499aa88fad9f58aad7c31832f,026c2f057932da75680b21ecdbd23bf9cb9350f3,028a310e23177c3747d37971678dd964ee28ce17,02a1be3a5dab38320f879489d8a1e0f2a72768b3,02d90a0bc95e068a8f75e00b3bd78579cb996c89,02d93bd28ad16b985328fa798b164c54bd8dcc0c,02eb2d69ac77b3bf6790139cb1a3b3c4571ecad4,02f764b7b09485106263e56ed9573966b48c0ad7,...,fd5da29ea31fc8fa2315f92f0c067e4e24e42b05,fd76579680a8142a19e69dcd6c33db61dc0a2444,fd7e06b88954f0ca0a2987e1741453624462bcc6,fd83a61cb82476f0ad6af28c4cd5872f285fd104,fd963f68e158c7e77bdd1d0a867927a1d270e972,fd977a3af7be241a9ed0213acb3aa75e5dc00253,fdb1ad87bd6fb08014267f2586faeed0edc7412b,fdc09ce75c596224ceb7e160e17c76f88453c246,fdc189e5a19850397f37201f4acc378cfddcf0d6,fdc19f011587b75c11a6c30d8ca06d90107b6bde,fdcce999b70500e2117e22cb9e1e5977ee28fbb3,fdf37fa13679f581bdfaae3b99e368633e0a144b,fdfad99711e053cd7dca1057f8dfb80b2f275728,fdfe926caf5f49a88a9bcab8d025e887f422128b,fe308dfc915f33c4217763c942ece65760cdf259,fe3211f90e4ab1f500e10fe175ae6142f4b13130,fe3461438b7a21c85a42ffd76030ece52e11dd7d,fe3c891bfb4d92361744aa697be30809c182020a,fe45b1591886927c2e9165618b4850019fd2606f,fe755bd56d9ed25f77b0d180a02c14e901af6332,fe994d2d43075b58afeee240f59149d3bed4660f,fe99e94b9d7235df4c50ec1b0ea9365e3ccdd5d3,feb15bdc09e48920ebf0b57460d7e70fe45c0553,feb165b7efdf8fdccb09368c512418917122f485,feb21e7aa4d21fba283d038b5cbf2ca33d929986,fec3a97cf01c9dfb1178693cc2924aabfbd4e1ef,ff079ef1efd3ec97d9914bc40e45aece8f680704,ff102b88d3012a2c798adee007c4a20172b0ffae,ff115962b9756008e3bd8b6a969d53b1ac7a4616,ff5c19eefe8884f125c3a91736fc0fb32ecdd919,ff70d33df144bdc870f2fd804d713e3c430f1c9b,ffa034716a62f0313b4cc2a05576e594b5ae7bdc,ffa41c79865d7fb336f586e0dec8b080db1027fb,ffc7c34369257431c7de9129094deb923bb3e3af,ffe53bd4dcfaa42668baf5ea0d2ddc676538fce0,fffedf27bf3bf8fa64d01e6b0a6affc9170716f1,x,y,f,path
0,4,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-92,-999,-999,-999,-999,-999,-999,-999,-999,-999,-74,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-51,-999,-999,-999,-999,-85,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,41.316772,180.01710,-1,5e1580bb1506f2000638fc62
1,4,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-92,-999,-999,-999,-999,-999,-999,-999,-999,-999,-72,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-54,-999,-999,-999,-999,-85,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,47.679752,178.77545,-1,5e1580bb1506f2000638fc62
2,4,-999,-999,-999,-999,-999,-999,-999,-999,

In [ ]:
for building in used_buildings:
    df = pd.read_csv(f'/content/drive/MyDrive/wifi_0_02/{building}_train.csv')
    print(f'{building}:{df.shape}')

5a0546857ecc773753327266:(9296, 3402)
5c3c44b80379370013e0fd2b:(9737, 3068)
5d27075f03f801723c2e360f:(23666, 7034)
5d27096c03f801723c31e5e0:(9100, 4969)
5d27097f03f801723c320d97:(10507, 2495)
5d27099f03f801723c32511d:(4251, 930)
5d2709a003f801723c3251bf:(3940, 1257)
5d2709b303f801723c327472:(15358, 1918)
5d2709bb03f801723c32852c:(17203, 2457)
5d2709c303f801723c3299ee:(10083, 5836)
5d2709d403f801723c32bd39:(10027, 2144)
5d2709e003f801723c32d896:(11042, 1314)
5da138274db8ce0c98bbd3d2:(2662, 495)
5da1382d4db8ce0c98bbe92e:(8999, 2867)
5da138314db8ce0c98bbf3a0:(9012, 1217)
5da138364db8ce0c98bc00f1:(2767, 827)
5da1383b4db8ce0c98bc11ab:(13196, 1530)
5da138754db8ce0c98bca82f:(7188, 1632)
5da138764db8ce0c98bcaa46:(9420, 1893)
5da1389e4db8ce0c98bd0547:(6312, 1026)
5da138b74db8ce0c98bd4774:(17382, 3540)
5da958dd46f8266d0737457b:(15148, 3504)
5dbc1d84c1eb61796cf7c010:(16174, 4524)
5dc8cea7659e181adb076a3f:(15655, 4869)


In [ ]:
a = pd.read_csv('/content/drive/MyDrive/fix_test_timestamp.csv')
a

,site_path_timestamp,real_timestamp
0,5a0546857ecc773753327266_046cfa46be49fc1083481...,1578474564146
1,5a0546857ecc773753327266_046cfa46be49fc1083481...,1578474573154
2,5a0546857ecc773753327266_046cfa46be49fc1083481...,1578474579463
3,5a0546857ecc773753327266_046cfa46be49fc1083481...,1578474582900
4,5a0546857ecc773753327266_046cfa46be49fc1083481...,1578474586465
...,...,...
10128,5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f...,1573731143307
10129,5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f...,1573731146476
10130,5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f...,1573731151613
10131,5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f...,1573731157617


In [ ]:
a_df = a["site_path_timestamp"].apply(lambda x: pd.Series(x.split("_")))

In [ ]:
a_df['timestamp'] = a['real_timestamp'].values

In [ ]:
a_df = a_df.rename({0:'site', 1:'path', 2:'timestamp', 'timestamp':'real_timestamp'}, axis=1)

In [ ]:
a_df

,site,path,timestamp,real_timestamp
0,5a0546857ecc773753327266,046cfa46be49fc10834815c6,0000000000009,1578474564146
1,5a0546857ecc773753327266,046cfa46be49fc10834815c6,0000000009017,1578474573154
2,5a0546857ecc773753327266,046cfa46be49fc10834815c6,0000000015326,1578474579463
3,5a0546857ecc773753327266,046cfa46be49fc10834815c6,0000000018763,1578474582900
4,5a0546857ecc773753327266,046cfa46be49fc10834815c6,0000000022328,1578474586465
...,...,...,...,...
10128,5dc8cea7659e181adb076a3f,fd64de8c4a2fc5ebb0e9f412,0000000082589,1573731143307
10129,5dc8cea7659e181adb076a3f,fd64de8c4a2fc5ebb0e9f412,0000000085758,1573731146476
10130,5dc8cea7659e181adb076a3f,fd64de8c4a2fc5ebb0e9f412,0000000090895,1573731151613
10131,5dc8cea7659e181adb076a3f,fd64de8c4a2fc5ebb0e9f412,0000000096899,1573731157617
